1. Provide investment suggestion according to questionnaire
The 'Investment Suggestion' function evaluates an investor's risk appetite through a questionnaire, then uses our scoring dictionary to provide customized investment advice, ensuring recommendations match the client's financial profile.

Step 1. Design questionnaire-V1:

Section 1: Personal Information
Age:
Under 30 [ ]
31-45 [ ]
46-60 [ ]
Over 60 [ ]

Annual Income:
Below $50,000 [ ]
$50,000 - $100,000 [ ]
$100,000 - $200,000 [ ]
Above $200,000 [ ]

Investment Time Horizon:
Less than 3 years [ ]
3-5 years [ ]
6-10 years [ ]
More than 10 years [ ]

Section 2: Investment Goals and Experience
Primary Investment Goal:
Capital preservation [ ]
Income [ ]
Balanced growth and income [ ]
Growth [ ]

Investment Experience:
None [ ]
Limited (e.g., savings accounts, CDs) [ ]
Moderate (e.g., stocks, bonds, mutual funds) [ ]
Extensive (e.g., options, futures) [ ]

Section 3: Risk Tolerance
Response to Market Downturn:
Sell all investments [ ]
Sell some [ ]
Do nothing [ ]
Buy more [ ]

Comfort Level with Investment Fluctuations:
Very uncomfortable [ ]
Somewhat uncomfortable [ ]
Neutral [ ]
Comfortable [ ]

Step 2. Build scoring dictionary to assign scores for client's answers:

In [2]:
scoring_dic = {
    "Section 1: Personal Information": {
        "Age": {
            "Under 30": 3,
            "31-45": 2,
            "46-60": 1,
            "Over 60": 0
        },
        "Annual Income": {
            "Below $50,000": 0,
            "$50,000 - $100,000": 1,
            "$100,000 - $200,000": 2,
            "Above $200,000": 3
        },
        "Investment Time Horizon": {
            "Less than 3 years": 0,
            "3-5 years": 1,
            "6-10 years": 2,
            "More than 10 years": 3
        }
    },
    "Section 2: Investment Goals and Experience": {
        "Primary Investment Goal": {
            "Capital preservation": 0,
            "Income": 1,
            "Balanced growth and income": 2,
            "Growth": 3
        },
        "Investment Experience": {
            "None": 0,
            "Limited": 1,
            "Moderate": 2,
            "Extensive": 3
        }
    },
    "Section 3: Risk Tolerance": {
        "Response to Market Downturn": {
            "Sell all investments": 0,
            "Sell some": 1,
            "Do nothing": 2,
            "Buy more": 3
        },
        "Comfort Level with Investment Fluctuations": {
            "Very uncomfortable": 0,
            "Somewhat uncomfortable": 1,
            "Neutral": 2,
            "Comfortable": 3
        }
    }
}

Step 3. Build an investment suggestion dictionary

In [3]:
investment_dic={
        "Low": {"Score Range": (0, 14), "Portfolio": "70% bonds, 20% stocks, 10% cash or cash equivalents"},
        "Medium": {"Score Range": (15, 28), "Portfolio": "50% stocks, 40% bonds, 10% alternative investments or cash"},
        "High": {"Score Range": (29, 42), "Portfolio": "80% stocks (including international and emerging markets), 15% bonds, 5% alternatives"}
    }

Step 4. Collect client's answers and calculate total score according to the scoring_dic
Then suggest investment suggetion according to the calculated scores and investment_dic

In [ ]:
##Pending for coding

2. Provide real-time stock performance information according to client's requirment
The 'Performance Update' delivers stock performance and financial insights. It utilizes the Yahoo Finance API for up-to-date information and presents it in a friendly format, aiding investors in making informed decisions based on the latest market data.

In [14]:
import pandas as pd
import requests
from pandas import json_normalize

1). Market information

In [16]:
# [Stock] market/v2/get-quotes

url = "https://apidojo-yahoo-finance-v1.p.rapidapi.com/market/v2/get-quotes"

querystring = {"region":"US","symbols":"AMD,IBM,AAPL"} #select three stocks

headers = {
	"X-RapidAPI-Key": "5907ff01a7msh24bd804afeffdf5p123becjsnc6c6b83b73b9",
	"X-RapidAPI-Host": "apidojo-yahoo-finance-v1.p.rapidapi.com"
}

response_quotes = requests.get(url, headers=headers, params=querystring)

# Check if the request was successful
if response_quotes.status_code == 200:
    # Convert the JSON content of the response to a dict
    json_data = response_quotes.json()
    
    # Assuming the relevant data is in the 'quoteResponse' -> 'result' part of the JSON
    # Adjust the path as per your JSON structure
    quotes_data = json_data['quoteResponse']['result']
    
    # Convert this data to a DataFrame
    df = pd.DataFrame(quotes_data)
    
    print(df)
else:
    print(f"Failed to fetch data: {response_quotes.status_code}")

  language region quoteType typeDisp         quoteSourceName  triggerable  \
0    en-US     US    EQUITY   Equity  Nasdaq Real Time Price         True   
1    en-US     US    EQUITY   Equity  Nasdaq Real Time Price         True   
2    en-US     US    EQUITY   Equity  Nasdaq Real Time Price         True   

  customPriceAlertConfidence  \
0                       HIGH   
1                       HIGH   
2                       HIGH   

                                        quoteSummary currency  \
0  {'earnings': {'maxAge': 86400, 'earningsChart'...      USD   
1  {'earnings': {'maxAge': 86400, 'earningsChart'...      USD   
2  {'earnings': {'maxAge': 86400, 'earningsChart'...      USD   

                                          components  ...  \
0  [^DWUSSR, ^GMB, ^GMBNTR, ^IXCO, ^IXIC, ^NQCRD,...  ...   
1  [^AXI, ^BGD, ^BGDNTR, ^CPQ, ^CPQNTR, ^DAA, ^DA...  ...   
2  [^AIX, ^AVSPY, ^CAPEXA, ^CAPEXAN, ^CPQ, ^CPQNT...  ...   

                                           pageViews  gm

In [19]:
# Seclect "Symbol", "quoteSummary", "currency" and "dividendRate" and store them separately
selected_columns_df = df[['symbol', 'currency', 'dividendRate']].copy()

# The 'quoteSummary' column contains nested JSON,extract it as a separate series 
quote_summary_series = df['quoteSummary'].copy()

print(selected_columns_df)

# [!Question] Normalize the first row's 'quoteSummary' data as an example
normalized_data = json_normalize(quote_summary_series.iloc[0])

print(normalized_data)

  symbol currency  dividendRate
0    AMD      USD           NaN
1    IBM      USD          6.64
2   AAPL      USD          0.96
   earnings.maxAge                   earnings.earningsChart.quarterly  \
0            86400  [{'date': '1Q2023', 'actual': 0.6, 'estimate':...   

   earnings.earningsChart.currentQuarterEstimate  \
0                                            0.6   

  earnings.earningsChart.currentQuarterEstimateDate  \
0                                                1Q   

   earnings.earningsChart.currentQuarterEstimateYear  \
0                                               2024   

  earnings.earningsChart.earningsDate  \
0            [1714474740, 1714996800]   

                     earnings.financialsChart.yearly  \
0  [{'date': 2020, 'revenue': 9763000000, 'earnin...   

                  earnings.financialsChart.quarterly  \
0  [{'date': '2Q2023', 'revenue': 5353000000, 'ea...   

  earnings.financialCurrency  
0                        USD  


In [10]:
# [markt] market/v2/get-summary

url = "https://apidojo-yahoo-finance-v1.p.rapidapi.com/market/v2/get-summary"

querystring = {"region":"US"} #US market

headers = {
	"X-RapidAPI-Key": "5907ff01a7msh24bd804afeffdf5p123becjsnc6c6b83b73b9",
	"X-RapidAPI-Host": "apidojo-yahoo-finance-v1.p.rapidapi.com"
}

market_summary = requests.get(url, headers=headers, params=querystring)

# Check if the request was successful
if market_summary.status_code == 200:
    # Convert the JSON content of the response to a dict
    json_data = market_summary.json()
    
    summary_data = json_data['marketSummaryAndSparkResponse']['result'] 
    # Convert this data to a DataFrame
    df = pd.DataFrame(summary_data)
    
    print(df)
else:
    print(f"Failed to fetch data: {market_summary.status_code}")



   exchangeTimezoneName fullExchangeName    symbol  gmtOffSetMilliseconds  \
0      America/New_York              SNP     ^GSPC              -14400000   
1      America/New_York              DJI      ^DJI              -14400000   
2      America/New_York      Nasdaq GIDS     ^IXIC              -14400000   
3      America/New_York  Chicago Options      ^RUT              -14400000   
4      America/New_York    NY Mercantile      CL=F              -14400000   
5      America/New_York            COMEX      GC=F              -14400000   
6      America/New_York            COMEX      SI=F              -14400000   
7         Europe/London              CCY  EURUSD=X                      0   
8      America/New_York      ICE Futures      ^TNX              -14400000   
9         Europe/London              CCY  GBPUSD=X                      0   
10        Europe/London              CCY     JPY=X                      0   
11                  UTC              CCC   BTC-USD                      0   

In [11]:
# Select useful columns
selected_columns_df = df[['exchangeTimezoneName', 'fullExchangeName', 'symbol', 'gmtOffSetMilliseconds', 'regularMarketTime', 'exchange', 'shortName', 'regularMarketPreviousClose']].copy()

# Converting 'regularMarketTime' and 'regularMarketPreviousClose' to more readable formats
# Since these columns contain nested data, extract the 'fmt' key for a human-readable format
selected_columns_df['regularMarketTime'] = selected_columns_df['regularMarketTime'].apply(lambda x: x.get('fmt') if isinstance(x, dict) else None)
selected_columns_df['regularMarketPreviousClose'] = selected_columns_df['regularMarketPreviousClose'].apply(lambda x: x.get('fmt') if isinstance(x, dict) else None)

print(selected_columns_df)


   exchangeTimezoneName fullExchangeName    symbol  gmtOffSetMilliseconds  \
0      America/New_York              SNP     ^GSPC              -14400000   
1      America/New_York              DJI      ^DJI              -14400000   
2      America/New_York      Nasdaq GIDS     ^IXIC              -14400000   
3      America/New_York  Chicago Options      ^RUT              -14400000   
4      America/New_York    NY Mercantile      CL=F              -14400000   
5      America/New_York            COMEX      GC=F              -14400000   
6      America/New_York            COMEX      SI=F              -14400000   
7         Europe/London              CCY  EURUSD=X                      0   
8      America/New_York      ICE Futures      ^TNX              -14400000   
9         Europe/London              CCY  GBPUSD=X                      0   
10        Europe/London              CCY     JPY=X                      0   
11                  UTC              CCC   BTC-USD                      0   

2). Stock information

In [13]:
# stock/v2/get-summary

url = "https://apidojo-yahoo-finance-v1.p.rapidapi.com/stock/v2/get-summary"

querystring = {"symbol":"AMRN","region":"US"} #one stock "AMRN"

headers = {
	"X-RapidAPI-Key": "5907ff01a7msh24bd804afeffdf5p123becjsnc6c6b83b73b9",
	"X-RapidAPI-Host": "apidojo-yahoo-finance-v1.p.rapidapi.com"
}

stock_summary = requests.get(url, headers=headers, params=querystring)

# Check if the request was successful
if stock_summary.status_code == 200:
    # Convert the JSON content of the response to a dict
    json_data1 = stock_summary.json()
    
    summary_profile = json_data1.get('summaryProfile', {})
    financial_data = json_data1.get('financialData', {})
    
    # Normalizing the extracted data into DataFrames
    summary_profile_df = pd.json_normalize(summary_profile)
    financial_data_df = pd.json_normalize(financial_data)
    
    print("Summary Profile:")
    print(summary_profile_df)
    print("\nFinancial Data:")
    print(financial_data_df)
else:
    print(f"Failed to fetch data: {stock_summary.status_code}")

Summary Profile:
                   industryDisp      zip                 industryKey  \
0  Drug Manufacturers - General  112-114  drug-manufacturers-general   

       sector  sectorDisp  fullTimeEmployees  \
0  Healthcare  Healthcare                275   

                                 longBusinessSummary    city            phone  \
0  Amarin Corporation plc, a pharmaceutical compa...  Dublin  353 0 166 99020   

   country   sectorKey companyOfficers                     website  maxAge  \
0  Ireland  healthcare              []  https://www.amarincorp.com   86400   

                       address1                      industry  \
0  Iconic Offices, The Greenway  Drug Manufacturers - General   

                                   address2  
0  Block C Ardilaun Court St Stephens Green  

Financial Data:
  recommendationKey financialCurrency  maxAge  ebitdaMargins.raw  \
0              hold               USD   86400           -0.04699   

  ebitdaMargins.fmt  profitMargins.raw profi

3. News

In [30]:
# news/v2/list
import requests

url = "https://apidojo-yahoo-finance-v1.p.rapidapi.com/news/v2/list"

querystring = {"region":"US","snippetCount":"28"}

payload = "Pass in the value of uuids field returned right in this endpoint to load the next page, or leave empty to load first page"
headers = {
	"content-type": "text/plain",
	"X-RapidAPI-Key": "5907ff01a7msh24bd804afeffdf5p123becjsnc6c6b83b73b9",
	"X-RapidAPI-Host": "apidojo-yahoo-finance-v1.p.rapidapi.com"
}

news = requests.post(url, data=payload, headers=headers, params=querystring)

if news.status_code == 200:
    json_data2 = news.json()

    news_data = json_data2.get('data', {}).get('main', {}).get('stream', [])

    if news_data:
        df_news = pd.DataFrame(news_data)
        print(df_news)
    else:
        print("No news data found.")
else:
    print(f"Failed to fetch data: {response.status_code}")
    

                                      id  \
0   4f29ac02-8725-3f8d-a2a1-5ae4e503d50a   
1   e077ede0-aced-3ac1-b2c1-0920ac478989   
2   d1108547-6209-3782-87af-abb3b5b93983   
3   7de401d6-e0ff-345c-8930-321355326acf   
4   5e96cb8a-720e-34d0-b0a4-980ad1c6c879   
5   e631a9db-ad5a-357a-87ea-e4c29a24bea7   
6   54849f4d-044a-3b28-a8e3-75a38e2601ef   
7   d9cb2aab-90fa-3a0f-abec-3b87ee621559   
8   cf3f905c-5f9f-3ebc-ad70-eda7f5c99a08   
9   0360ed92-febc-3cfd-8bda-03faf36301ac   
10  3ec1d5b8-fa65-3f4c-a3d3-9c39f1c593ba   
11  4b40e7c8-7c33-35c0-991d-5e7d7779b5bb   
12  760c7832-56d6-3724-b34c-f59b48043b49   
13  4572a5cb-d6b8-3445-a212-a6eac63d6c70   
14  f655ceff-f064-385e-9ccd-3d1322ff1106   
15  ad1122c9-9091-327d-a5d5-ab02cd5c3f21   
16  a085ae57-66d7-39e7-9dca-0d0853f436ff   
17  85bfed3c-4ff0-360f-961c-4056d354ade9   
18  5eaab9f8-9458-3afa-970b-a4bda8075e61   
19  01ce10f6-3cef-3057-a712-4b561043a98f   
20  dc0109ea-9389-33ce-83be-ee43bc60d486   
21  e902f44f-81a4-3a67-a993-994d

In [31]:
# print first record content
first_record_content = df_news.loc[0, 'content']  # This extracts the content dictionary for the first record
print(first_record_content)

first_record = json_normalize(first_record_content)

print(first_record)


{'id': '4f29ac02-8725-3f8d-a2a1-5ae4e503d50a', 'contentType': 'STORY', 'title': 'Analyst adjusts stock price target for Nvidia ahead of conference', 'pubDate': '2024-03-13T16:46:45Z', 'tags': ['yfrontpage:region=us', 'ymedia:signal=summary_source:provider', 'ymedia:provider_id=thestreet_881', 'ymedia:provider_category=Stocks', 'ymedia:provider_category=Investing', 'ymedia:category=fin_whitelistedproviders', 'ymedia:hosted=yes', 'ymedia:provider_category=Artificial Intelligence', 'ymedia:provider_category=Nvidia', 'ymedia:provider_category=EARNINGS', 'ymedia:provider_category=Price Target', 'ymedia:provider_category=TECHNOLOGY', 'ymedia:category=100000002', 'ymedia:provider_category=Earnings', 'ymedia:provider_category=INVESTING', 'ymedia:provider_category=Software & Services', 'ymedia:category=fin_tier2', 'ymedia:provider_category=Analyst Upgrade', 'ymedia:category=100000008', 'ymedia:category=100000007', 'ymedia:provider_category=Software', 'ymedia:provider_category=Technology', 'ymed

3. Send "Daily news" to subscribed investors
The 'Daily News' is our proactive communication feature. It sends subscribers a daily briefing of stock performance and capital market news. This feature compiles data into a concise PDF report and distributes it via automated emails, keeping our clients informed with minimal effort.

In [ ]:
# Pending for coding